# 0. Global Items

## 0.1 Imports and Constants

In [2]:
# Imports

import numpy as np
from matrix_helper import *

# Constants

PRIME = 769

## 0.2 Functions

In [3]:
# Functions that do not use global variables

def dtb(base, number):
    """ Converts the number to a base representation """
    test = number
    count = 0
    
    # Find the larget power of base in the number
    while test >= base:
        test = test/base
        count += 1
    answer = 0
    
    # Converts to our usual base 10 representation
    while count >= 0:
        test = number//pow(base,count)
        answer = 100*answer + test
        number = number - test*pow(base,count)
        count -= 1
    return answer

In [8]:
# Functions that use global variables

def hash_matrix(matrix):
    """ Creates a hash of the matrix values. """
    length = list(range(len(matrix)))
    hashed = 0
    
    # Hash is based on value and placement
    for column in length:
        for row in length:
            hashed = PRIME*hashed + (matrix[row,column])*(row+1)*(column+1)
        
    return hashed

def matrices_equal(first, second):
    if first[0] != second[0]:
        return False
    elif (first[1] == second[1]).all():
        return True
    else:
        return False
    
def cnot_replace_toffoli(permute):
    """ Uses permute to check if a matrix has been seen before.
    
    Takes the list in permute, pops off the value at the end, if the matrix specified by value
    is in CHECKED_COMBINATIONS, then move onto the next value in permute. If not, add the new 
    permutations of this to the list, corresponding to the quantum circuit with an extra cnot added.
    """
    
    while permute != []:
        # Pop off the next value
        curr_val = permute.pop()

        # Compute the matrix
        curr_matrix = produce_matrix(curr_val)
        hashed_matrix = hash_matrix(curr_matrix)

        # If it is a toffoli, then return the value
        if matrices_equal([hashed_matrix, curr_matrix], [TOFFOLI_HASH, TOFFOLI]):
            print("A combination of CNOTs that could replace a Toffoli was found.")
            return curr_val[0]

        # Check if it is the same as a previous one, if not same add in permutations
        elif not same_as_previous(curr_matrix, hashed_matrix):
            CHECKED_COMBINATIONS.append((curr_val[0], curr_matrix, hashed_matrix))
            new_permute = add_permute(curr_val[0])
            new_permute.extend(permute)        
            permute = new_permute
    
    # If empty stop and return unsuccessful
    print("A combination of CNOTs that could replace a Toffoli was not found.")
    print(len(CHECKED_COMBINATIONS),"unique combinations were found.")
    return None
    
def produce_matrix(value):
    """ Find the matrix specified by value 
    
    Args:
        value (list)
            value[0]: The current value which specifies the matrix of n gates
            value[1]: The previous value that specified the matrix of n-1 gates
    """
    
    # If it is one of the original gates
    if value[0] < BASE:
        return CNOT_MAPPING[value[0]]
    
    # Otherwise, find out what new gate is specified
    index = dtb(BASE, value[0])
    while index >= 100:
        index = int(index/100)
    
    # Return the new gate right multiplied by the past matrix
    return np.matmul(CNOT_MAPPING[index], find_matrix(value[1]))

def find_matrix(value):
    """ Finds the matrix of n-1 gates of value from CHECKED_COMBINATIONS """

    # Use CHECKED_COMBINATIONS to find the previous matrix used
    for val_matrix in CHECKED_COMBINATIONS:
        prev_val = val_matrix[0]
        if prev_val == value:
            return val_matrix[1]

def same_as_previous(matrix, hashed):
    """ Checks if matrix has been seen in CHECKED_COMBINATIONS """
    
    # For each value/matrix pair that has been computed
    for val_matrix in CHECKED_COMBINATIONS:
        # Check if our current matrix has all entries equal
        if matrices_equal([val_matrix[2], val_matrix[1]],[hashed, matrix]):
            return True
    return False

def add_permute(value):
    """ Adds the next gate permutation to value """
    
    # Find the correct largest power
    index = dtb(BASE, value)
    count = 1
    while index >= 100:
        index = int(index/100)
        count += 1
    
    # Add the correct power and create all the permutations
    final = []
    for i in range(BASE - 1):
        i += 1
        final.append([i*pow(BASE, count)+value,value])
    return final

In [9]:
# Functions for checking permutations correctness

def check_duplicates(test_list):
    """ Checks if test_list has duplicate matrices """
    while len(test_list) > 0:
        testing = test_list.pop()
        for existing in test_list:
            if np.all(existing[1] == testing[1]):
                print("A duplicate was found:")
                print_matrix(existing[1])
                print()
                print_matrix(testing[1])

def check_correctness(test_list, size):
    """ Checks if test_list has matrices with the correct representation """
    
    # For each value/matrix pair in CHECKED_COMBINATIONS
    for m in test_list:

        # Get the value in a base representation, as well as the matrix
        val = dtb(BASE, m[0])
        matrix = m[1]
        val_matrix = identity_matrix(size)

        # Reproduce the matrix specified by val
        while val >= 1:
            val_matrix = np.matmul(CNOT_MAPPING[val % 100], val_matrix)
            val = int(val/100)

        # If not equal to the other matrix, note it down
        if not (val_matrix == matrix).all():
            print(dtb(BASE, m[0]), "is incorrect")

# 1. No Ancilla Permutations

## 1.1 Constants

In [10]:
BASE = 7 # Base 7 representation
CHECKED_COMBINATIONS = [] # Will hold [value, matrix] pairs
CNOT_MAPPING = [ # Use the index to refer to the matrix
    None, # Using 0 leads to issues in representation so indexing starts from 1
    np.matrix(step_matrix(0, cnot([1],2), 1)),
    np.matrix(cnot([1],3)),
    np.matrix(step_matrix(0, cnot([2],1), 1)),
    np.matrix(cnot([2],3)),
    np.matrix(cnot([3],1)),
    np.matrix(cnot([3],2)),
]
TOFFOLI = np.matrix(cnot([1, 2],3))
TOFFOLI_HASH = hash_matrix(TOFFOLI)

<ipython-input-8-a32f0f0ffcde>:11: RuntimeWarning: overflow encountered in long_scalars
  hashed = PRIME*hashed + (matrix[row,column])*(row+1)*(column+1)


## 1.2 Finding Unique Combinations

In [11]:
test_list = []
for i in list(range(BASE-1)):
    i += 1
    test_list.append((BASE-i, BASE-i, hash_matrix(CNOT_MAPPING[BASE-i])))

# Run the checking function
cnot_replace_toffoli(test_list)

<ipython-input-8-a32f0f0ffcde>:11: RuntimeWarning: overflow encountered in long_scalars
  hashed = PRIME*hashed + (matrix[row,column])*(row+1)*(column+1)


A combination of CNOTs that could replace a Toffoli was not found.
168 unique combinations were found.


## 1.3 Printing Combinations

In [12]:
# Print all combination values in decimal, by 100s

print("All outcomes:")
for i in CHECKED_COMBINATIONS:
    print(dtb(BASE, i[0]),end=", ")

All outcomes:
1, 2, 3, 4, 5, 6, 601, 501, 401, 301, 201, 101, 602, 502, 402, 302, 603, 503, 403, 203, 103, 604, 504, 104, 605, 405, 205, 105, 406, 306, 206, 40601, 30601, 40501, 30501, 20501, 60401, 50401, 30401, 60301, 20301, 10301, 50201, 30201, 50602, 40602, 30602, 40502, 30502, 20502, 10502, 60402, 50402, 60302, 10302, 40603, 20603, 10603, 20503, 10503, 60403, 10403, 60203, 10203, 50103, 50604, 40604, 30604, 10604, 20504, 10504, 50104, 40605, 20605, 20405, 10405, 10205, 40105, 30406, 20406, 20306, 30206, 6040601, 5040601, 3040601, 2030601, 1030601, 6040501, 2040501, 1040501, 2030501, 1030501, 5020501, 3020501, 5060401, 1050401, 6030401, 1030401, 2060301, 1060301, 6020301, 5020301, 1020301, 4010301, 3050201, 6030201, 4050602, 2050602, 6040602, 5040602, 1030602, 6040502, 2040502, 1040502, 2030502, 1030502, 3020502, 1020502, 4010502, 2060302, 1060302, 6040603, 2040603, 1040603, 4010603, 6020503, 1020503, 4010503, 5010403, 5010203, 4050103, 4050604, 2050604, 3040604, 2030604, 4010604, 

## 2.4 Checking Duplicates

In [13]:
# No output means no duplicates found

check_duplicates(CHECKED_COMBINATIONS[:])

## 2.5 Checking Correctness

In [14]:
# No output means no errors found

check_correctness(CHECKED_COMBINATIONS[:], 8)

# 2. One Ancilla Permutations

## 2.1 Constants

In [59]:
# Constants

BASE = 10 # Base representation
CHECKED_COMBINATIONS = [] # Will hold [value, matrix] pairs
CNOT_MAPPING = [ # Use the index to refer to the matrix
    None, # Using 0 leads to issues in representation so indexing starts from 1
    np.matrix(step_matrix(0, cnot([1],2), 2)),
    np.matrix(step_matrix(1, cnot([1],2), 1)),
    np.matrix(step_matrix(2, cnot([1],2), 0)),
    np.matrix(step_matrix(0, cnot([2],1), 2)),
    np.matrix(step_matrix(1, cnot([2],1), 1)),
    np.matrix(step_matrix(2, cnot([2],1), 0)),
    np.matrix(step_matrix(0, cnot([1],3), 1)),
    np.matrix(step_matrix(1, cnot([1],3), 0)),
    np.matrix(step_matrix(0, cnot([1],4), 0)),
    np.matrix(step_matrix(0, cnot([3],1), 1)),
    np.matrix(step_matrix(1, cnot([3],1), 0)),
    np.matrix(step_matrix(0, cnot([4],1), 0)),
]
TOFFOLI = np.matrix(step_matrix(0, cnot([1, 2], 3), 1))
TOFFOLI_HASH = hash_matrix(TOFFOLI)

<ipython-input-44-750c2a3a434a>:11: RuntimeWarning: overflow encountered in long_scalars
  hashed = PRIME*hashed + (matrix[row,column])*(row+1)*(column+1)


## 2.2 Finding Unique Combinations

In [60]:
# Create the list of just 1 gate
test_list = []
for i in list(range(BASE-1)):
    i += 1
    test_list.append([BASE-i, BASE-i])

# Run the checking function
cnot_replace_toffoli(test_list)

8
16
24
32
40
48
56
64
72
80
88
96
104
112
120
128
136
144
152
160
168
176
184
192
200
208
207
215
223
231
239
247
255
263
262
270
269
277
285
293
301
309
308
307
306
314
322
330
338
346
345
353
352
360
359
367
375
383
382
390
389
397
396
395
403
411
410
409
417
425
433
432
431
439
438
437
445
453
452
451
450
458
457
456
455
463
462
470
469
468
467
466
465
473
481
489
497
505
513
512
511
510
518
526
534
542
550
558
557
556
555
554
562
561
569
577
576
575
583
591
590
589
597
605
613
621
620
619
627
626
634
633
641
649
657
656
664
663
671
670
678
677
685
693
701
700

<ipython-input-44-750c2a3a434a>:11: RuntimeWarning: overflow encountered in long_scalars
  hashed = PRIME*hashed + (matrix[row,column])*(row+1)*(column+1)



699
707
715
714
713
712
720
719
718
717
716
715
723
722
730
729
728
727
726
725
724
723
722
721
720
719
718
726
734
742
750
758
766
765
773
772
771
779
778
786
794
793
792
800
799
798
797
805
813
812
820
819
818
826
825
824
823
831
839
847
846
854
853
861
860
868
867
875
883
891
899
907
906
905
904
912
911
919
918
926
925
924
932
940
939
938
937
936
944
943
951
950
949
948
947
946
945
944
943
951
950
958
966
974
973
981
980
979
978
986
994
993
1001
1000
999
1007
1006
1005
1004
1012
1020
1028
1027
1035
1034
1042
1041
1040
1039
1047
1055
1063
1071
1070
1069
1068
1076
1084
1083
1091
1090
1098
1097
1105
1104
1112
1111
1119
1118
1117
1125
1133
1132
1131
1130
1129
1128
1127
1126
1125
1133
1132
1131
1139
1147
1155
1154
1153
1152
1160
1159
1158
1166
1174
1182
1181
1180
1179
1187
1186
1185
1184
1192
1200
1199
1207
1206
1214
1222
1230
1229
1228
1236
1235
1243
1242
1250
1249
1257
1265
1273
1272
1280
1279
1287
1295
1294
1302
1301
1300
1299
1298
1297
1305
1304
1303
1311
1319
1327
1326
1334
1333
13

KeyboardInterrupt: 

## 2.3 Printing Combinations

In [29]:
# Print all combination values

print("All outcomes:")
for i in CHECKED_COMBINATIONS:
    print(dtb(BASE, i[0]),end=", ")

All outcomes:
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 1201, 1101, 1001, 901, 801, 701, 601, 501, 401, 301, 201, 101, 1202, 1102, 1002, 902, 802, 702, 602, 502, 402, 302, 102, 1203, 1103, 1003, 903, 803, 703, 603, 503, 403, 203, 1204, 1104, 1004, 904, 804, 704, 604, 504, 104, 1205, 1105, 1005, 905, 805, 705, 605, 405, 205, 1206, 1106, 1006, 906, 806, 706, 506, 306, 1207, 1107, 1007, 907, 807, 507, 407, 307, 1208, 1108, 1008, 908, 608, 508, 108, 1209, 1109, 1009, 609, 409, 1210, 1110, 910, 710, 610, 210, 110, 1211, 911, 811, 411, 311, 211, 912, 812, 712, 312, 112, 101201, 91201, 81201, 71201, 61201, 51201, 41201, 31201, 21201, 101101, 81101, 71101, 61101, 51101, 41101, 31101, 21101, 91001, 81001, 71001, 61001, 41001, 31001, 21001, 120901, 100901, 70901, 60901, 50901, 40901, 30901, 20901, 120801, 110801, 70801, 60801, 50801, 40801, 30801, 20801, 120701, 100701, 60701, 40701, 30701, 100601, 90601, 80601, 50601, 40601, 30601, 20601, 80501, 60501, 40501, 30501, 20501, 110401, 90401, 70401, 50

## 2.4 Checking Duplicates

In [116]:
# No output means no duplicates found

check_duplicates(CHECKED_COMBINATIONS[1:3])

## 2.5 Checking Correctness

In [115]:
# No output means no errors found

check_correctness(CHECKED_COMBINATIONS[1:3], 16)

In [33]:
PRIME = 769

def hash_matrix(matrix):
    length = list(range(len(matrix)))
    hashed = 0
    for column in length:
      for row in length:
        hashed = PRIME*hashed + (matrix[row,column])*(row+1)*(column+1)
    return hashed

print(hash_matrix(CHECKED_COMBINATIONS[2][1]))
print(hash_matrix(CHECKED_COMBINATIONS[2][1]))
# print(np.sum(hash_matrix(CHECKED_COMBINATIONS[2][1])) == np.sum(hash_matrix(CHECKED_COMBINATIONS[1][1])))
print(np.sum(hash_matrix(CHECKED_COMBINATIONS[1][1])))
print(np.sum(hash_matrix(CHECKED_COMBINATIONS[0][1])))

print(len(CHECKED_COMBINATIONS))

-3094843935434593836
-3094843935434593836
7937632894156256712
5349569340700263832
13113


<ipython-input-33-a0b28b9b0577>:8: RuntimeWarning: overflow encountered in long_scalars
  hashed = PRIME*hashed + (matrix[row,column])*(row+1)*(column+1)
